In [2]:
import numpy as np
import statistics as st
import scipy.stats as stats
import matplotlib.pyplot as plt

### Problem 1

In [ ]:
# read file
with open('data/abalone.data') as f:
     read_data = f.read()
read_data = read_data.split('\n')

In [ ]:
# load string into data
data = []
for str in read_data:
    for entry in str.split(','):
        if entry == 'M':
            data.append(0.0)
        elif entry == 'F':
            data.append(1.0)
        elif entry == 'I':
            data.append(2.0)
        else:
            if len(entry) > 0:
                data.append(float(entry))

# convert data to numpy arrays and reshape
data = np.array(data).reshape((-1, 9))

In [ ]:
n = 5   # number of samples
r = 7000   # number of repetition
pop_median = np.median(data[:, 1])

# randomly select a number of samples from the data
def draw(sample_size):
    return np.random.choice(data[:, 1], sample_size, replace = False)   # 1 is index for Length

# calculate the confidence interval using stderr
z_upper = stats.norm.ppf(0.95)
z_lower = stats.norm.ppf(0.05)
def confidence_interval(array):
    stderr = st.stdev(array)
    mean = array.mean()
    upper = mean + z_upper * stderr
    lower = mean + z_lower * stderr
    return lower, upper

# plot sample medians
def plot_sample(medians):
    plt.hist(medians, 100, color ='green',  alpha = 0.7) 
    plt.xlabel('Medians') 
    plt.ylabel('Number of samples') 
    plt.title('The medians generated by one Bootstrap process\n', fontweight ="bold")  
    plt.show() 
    print(medians.size)

# run experiment to draw sample N times from the data
def experiment(sample_size, plot=True):
    count = 0.0
    for j in range(n):
        sample = draw(sample_size)
        medians = np.zeros(r)
        for i in range(r):
            # replicate r samples from sample and append median in to medians
            medians[i] = np.median(np.random.choice(sample, sample_size, replace = True))
        if plot:
            plot = False
            plot_sample(medians)
        lower, upper = confidence_interval(medians)
        if lower <= pop_median and upper >= pop_median:
            count += 1
        if j % 100 == 0:
            print(j, " runs")
    print("There are ", count/n, "of samples contain the true median in their interval")

In [ ]:
experiment(10)

In [ ]:
experiment(100)

### Probelm 6

In [16]:
# read file
with open('data/adult.data') as f:
     read_data = f.read()
read_data = read_data.split('\n')

In [34]:
# load string into data
edu = []
gender = []
income = []
for str in read_data:
    entries = str.split(',')
    if len(entries) < 10:
        break
    edu.append(entries[3][1:])
    if (entries[9] == " Male"):
        gender.append(1)
    else:
        gender.append(0)
    if (entries[-1] == " >50K"):
        income.append(1)
    else:
        income.append(0)
size = len(edu)
gender = np.array(gender)
income = np.array(income)

In [40]:
# test independence between gender and income
inc_50 = np.count_nonzero(income==1)
male_50 = np.count_nonzero(gender[income==1]) / inc_50
female_50 = 1 - male_50
inc_0 = np.count_nonzero(income==0)
male_0 = np.count_nonzero(gender[income==0]) / inc_0
female_0 = 1 - male_0

male = np.count_nonzero(gender) / gender.size
female = 1 - male

chi2 = (male-male_50)**2/male + (male-male_0)**2/male + (female-female_50)**2/female + (female-female_0)**2/female
chi2

0.16185964644413728

In [51]:
d_50 = {}
d_0 = {}
# initiate dict
for i in range(size):
    d_50[edu[i]] = 0.0
    d_0[edu[i]] = 0.0
for i in range(size):
    if income[i] == 1:
        d_50[edu[i]] += 1.0
    else:
        d_0[edu[i]] += 1.0

In [56]:
# test independence between edu and income
pop = 0.0
for i in d_50:
    pop += d_50[i] + d_0[i]
chi2 = 0.0
for i in d_50:
    edu_pop = d_50[i] + d_0[i]
    theory = edu_pop / pop
    chi2 += (theory - d_50[i] / edu_pop)**2 / theory + (theory - d_0[i] / edu_pop)**2 / theory
chi2

1234.2446663405835

In [57]:
count = 0
for i in d_0:
    count += 1
count

16